## Distortion correction with Au SAED data 
Adapted from Tina Bergh


In [84]:
# Activate interactive plotting
%matplotlib qt 

In [3]:
# Import packages
import numpy as np
import hyperspy.api as hs # v1.7.1
import matplotlib.pyplot as plt 
import skimage as ski
import pyxem as pxm  # v0.14.2 dependent on Pint v0.19.1

from pyxem.generators.calibration_generator import CalibrationGenerator
from scipy.signal import argrelextrema

from hyperspy.misc.utils import stack as stack_method
from numpy import log10

from scipy.spatial import distance_matrix

In [165]:
########################
# Measurements on SAED patterns:
########################

file = 'AuX_80kV_200cm'
folder = 'D:/Aurora/Master data/DataFebruar/240215_AuX_80kV/' 

# Intial loading of .mib file
file_ending = '.mib'
saed = pxm.load_mib(folder + file + file_ending)
saed.compute()
saed = pxm.signals.ElectronDiffraction2D(saed).inav[0]
saed.save(folder+file+'_s.hdf5')


This mib file appears to be TEM data. The stack is returned with no reshaping.
[########################################] | 100% Completed | 109.63 ms


In [166]:
# Set center of direct beam 
scale = 1
saed.set_diffraction_calibration(scale)
saed.plot(cmap='magma_r', norm='log')

roi = hs.roi.CircleROI(cx=0, cy=0, r=18, r_inner=0)
roi.add_widget(saed)

saed.save(folder+file+'_center.hdf5')

In [167]:

cal = CalibrationGenerator(diffraction_pattern=saed)

# Plot saed diffraction pattern
saed.plot(norm="log") 
# Adjust x1, y1, x2, y2 according to detector used
line = hs.roi.Line2DROI(x1=-256, y1=-256, x2=256, y2=256, linewidth=5) 
line.add_widget(saed)

# Obtain line trace (here set to go through center of beam)
trace = line(saed)
trace = trace.as_signal1D(0)
trace.plot()

In [168]:
#############################################
# Mask out direct beam 
radius = 32 
ssum_m = saed.copy()
dbm = ssum_m.get_direct_beam_mask(radius=radius)
ssum_m.data = ssum_m.data * ~dbm.data
ssum_m.plot(norm="log")

4 mask_length


In [175]:
# Calculate distortion 
#   Mask radius, spread and asymmetry might have to be edited
#   Larger cameralength required higher mask_radius
spread =  2 
distortion = cal.get_elliptical_distortion(mask_radius=140, 
                                           scale=100, amplitude=1000,
                                           asymmetry=1.1, spread=spread)  
np.set_printoptions(precision=5, suppress=True)
print(cal.affine_matrix)
print(cal.ring_params)


c:\Users\Windows User\anaconda3\envs\masterENV\lib\site-packages\diffsims\utils\ring_pattern_utils.py:29: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt((xp - xcp) ** 2 + asym * (yp - ycp) ** 2)


[[1.02096 0.02094 0.     ]
 [0.02094 1.02091 0.     ]
 [0.      0.      1.     ]]
[  476.24018  3262.71123    23.87949 42098.28419     1.0855      0.78603]


In [170]:
# Find scale and plot residuals
mask_length = radius # tror denne må være lik radiusen
linewidth = 5
scale = cal.get_diffraction_calibration(mask_length=mask_length, linewidth=linewidth) 
print('scale=' + '{:.5f}'.format(scale))

residuals = cal.get_distortion_residuals(mask_radius=radius, spread=spread)
residuals.plot(cmap='RdBu', vmax=0.04) 


[########################################] | 100% Completed | 121.76 ms
[########################################] | 100% Completed | 265.10 ms
[########################################] | 100% Completed | 107.04 ms
[########################################] | 100% Completed | 125.39 ms
scale=0.00327


c:\Users\Windows User\anaconda3\envs\masterENV\lib\site-packages\diffsims\utils\ring_pattern_utils.py:102: RuntimeWarning: divide by zero encountered in power
  v.append(direct_beam_amplitude * Ri ** -2)  # np.exp((-1*(Ri)*(Ri))/d0)


[########################################] | 100% Completed | 119.16 ms
[########################################] | 100% Completed | 119.48 ms
[########################################] | 100% Completed | 108.07 ms


In [171]:
# Plot corrected diffraction pattern
cal.plot_corrected_diffraction_pattern(norm="log")


[########################################] | 100% Completed | 156.00 ms
[########################################] | 100% Completed | 109.14 ms


In [172]:
# Save the distortion matrix
np.savetxt(folder + file + '_dist.txt', distortion)


In [173]:
# MANUAL calibration of Au rings

# Inverse d spacings for gold
ids = [0.424522257, 0.490196078, 0.693241942, 0.812898233, 0.849460916, 0.980872977]

size = ssum_m.data.shape[0]
dpegs = stack_method([saed, saed, saed, saed])
dpegs = pxm.signals.ElectronDiffraction2D(dpegs.data.reshape((2, 2, size, size)))

dpegs = dpegs.apply_affine_transformation(
    cal.affine_matrix, preserve_range=True, inplace=False)
dpegm = dpegs.inav[0, 0]

dpegm.set_diffraction_calibration(1)
dpegm.plot(norm='log')


[########################################] | 100% Completed | 106.90 ms
[########################################] | 100% Completed | 120.70 ms


In [174]:

# Radius of Au rings to be plotted with the Au-pattern. Observe if the rings overlap, if not, adjust the radius.
# 30cm: 18.5, 21.5, 31.5, 37.5
# 40cm: 23.5, 27.5, 39.5, 46.5
# 50cm: 31.5, 34.5, 51.5, 58.5
# 60cm: 37.5, 43.5, 62.5, 73.5
# 80cm: 51.5, 58.5, 84.5, 99.5
# 100cm: 59.5, 69.5, 98.5, 115.5
# 120cm: 71.5, 84.5, 119.5, 139.5
# 150cm:86.5, 100.5, 141.5, 165.5
# 200 cm: 106.5, 123.5, 175.5, 205.5

roi111 = hs.roi.CircleROI(cx=0, cy=0, r=106.5)
roi111.add_widget(dpegm)
roi002 = hs.roi.CircleROI(cx=0, cy=0, r=123.5)
roi002.add_widget(dpegm)
roi022 = hs.roi.CircleROI(cx=0, cy=0, r=175.5)
roi022.add_widget(dpegm)
roi113 = hs.roi.CircleROI(cx=0, cy=0, r=205.5)
roi113.add_widget(dpegm)


In [163]:
# Calculate mean of manually calculated scales
scales = np.array([ids[0] / roi111.r, ids[1] / roi002.r, ids[2] / roi022.r, ids[3] / roi113.r])
print(scales, np.std(scales), np.mean(scales))
calibration = np.mean(scales)
print(calibration)

[0.00491 0.00488 0.0049  0.00491] 1.3221264179707376e-05 0.00489908820311879
0.00489908820311879


In [164]:

# Save calibration scale and figure
np.savetxt(folder + file + '_calibration_manual.txt', [calibration])

plt.savefig(folder+file + '_cal_rings.png')



In [176]:

# Save png figure of shift
diff = dpegs.inav[0, 0].data - ssum_m.data
if np.any(diff < 0.0):
    diff = diff + abs(np.min(diff.data))

diff = pxm.signals.ElectronDiffraction2D(diff)
diff_m = diff.copy()
dbm_ = diff_m.get_direct_beam_mask(radius=15)
diff_m.data = diff_m.data * ~dbm_.data
diff_m.plot(norm='log', scalebar=False, colorbar=False, axes_ticks=False)
plt.savefig(folder + file + '_diff.png')
# Make tif
diff_img = diff_m.copy()
diff_img.data = diff_img.data / diff_img.data.max()
diff_img.data = ski.img_as_uint(diff_img.data)
diff_img.save(folder + file + '_diff.tif')
shift = diff_m.data[diff_m.data > 0].min()
# Make log tif
diff_log = diff_m.copy()
diff_log.data = log10(diff_m.data + shift) - log10(shift)
diff_log.data = diff_log.data / diff_log.data.max()
diff_log.data = ski.img_as_uint(diff_log.data)
diff_log.save(folder + file + '_diff_log.tif') 